# Time Series with LSTMs

Cassandra Maldonado

1. Import, clean and plot the input data.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
import tensorflow as tf
import os
from helper import timeseries_train_test_split

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
def import_and_clean_data(filename='DAYTON_hourly.csv'):
    df = pd.read_csv(filename)
    
    # Datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    
    # Index
    df.set_index('Datetime', inplace=True)
    
    # Checking for missing values
    print(f"Missing values: {df.isna().sum()}")
    
    # Time series plot
    plt.figure(figsize=(14, 6))
    plt.plot(df['DAYTON_MW'])
    plt.title('Dayton Power Consumption (MW)')
    plt.xlabel('Date')
    plt.ylabel('Power (MW)')
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    return df

# LSTM sequence
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

# LSTM model
def build_and_train_lstm(X_train, y_train, X_test, y_test, bidirectional=False, epochs=50, batch_size=32):
    # Reshape input data for LSTM [samples, time steps, features]
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
    
    model = Sequential()
    
    if bidirectional:
        model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(X_train.shape[1], 1)))
    else:
        model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], 1)))
    
    model.add(Dense(1))
    
    model.compile(optimizer='adam', loss='mse')
    
    # Training 
    history = model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_test, y_test),
        verbose=1
    )
    
    # Predicting
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)
    
    # RMSE
    train_rmse = np.sqrt(np.mean((train_predictions - y_train) ** 2))
    test_rmse = np.sqrt(np.mean((test_predictions - y_test) ** 2))
    
    print(f"Train RMSE: {train_rmse:.2f}")
    print(f"Test RMSE: {test_rmse:.2f}")
    
    return model, history, test_predictions, test_rmse

# Model predictions
def plot_predictions(y_test, predictions, title):
    plt.figure(figsize=(14, 6))
    plt.plot(y_test, label='Actual')
    plt.plot(predictions, label='Predicted')
    plt.title(title)
    plt.xlabel('Time Step')
    plt.ylabel('Power (MW)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def main():
    # 1. Import and clean the data
    df = import_and_clean_data()
    
    # Extract the target column
    data = df['DAYTON_MW'].values


2. Train test split (use provided helper function helper.py Download helper.py, as we can’t use sklearn’s for time series data).

In [ ]:
train_size = int(len(data) * 0.8)
    train_data, test_data = timeseries_train_test_split(data, train_size)
    
    print(f"Training data size: {len(train_data)}")
    print(f"Testing data size: {len(test_data)}")

3. Fit a single layer LSTM with lag = 3 hours.

In [ ]:
print("Single Layer LSTM with lag = 3 hours")
    lag_3 = 3
    X_train_3, y_train_3 = create_sequences(train_data, lag_3)
    X_test_3, y_test_3 = create_sequences(test_data, lag_3)
    
    model_3, history_3, predictions_3, rmse_3 = build_and_train_lstm(
        X_train_3, y_train_3, X_test_3, y_test_3
    )
    
plot_predictions(y_test_3, predictions_3, 'Single Layer LSTM (lag=3) Predictions')

4. Fit a single layer LSTM with lag = 24 hours.

In [ ]:
print("Single Layer LSTM with lag = 24 hours")
    lag_24 = 24
    X_train_24, y_train_24 = create_sequences(train_data, lag_24)
    X_test_24, y_test_24 = create_sequences(test_data, lag_24)
    
    model_24, history_24, predictions_24, rmse_24 = build_and_train_lstm(
        X_train_24, y_train_24, X_test_24, y_test_24
    )
    
plot_predictions(y_test_24, predictions_24, 'Single Layer LSTM (lag=24) Predictions')

5. Fit a bidirectional LSTM with lag = 24 hours.

In [ ]:
print("\n=== Bidirectional LSTM with lag = 24 hours ===")
    model_bi, history_bi, predictions_bi, rmse_bi = build_and_train_lstm(
        X_train_24, y_train_24, X_test_24, y_test_24, bidirectional=True
    )
    
plot_predictions(y_test_24, predictions_bi, 'Bidirectional LSTM (lag=24) Predictions')

6. Plot results on test set and compare performance using RMSE.

In [ ]:
print("\n=== Performance Comparison (RMSE) ===")
print(f"Single Layer LSTM (lag=3): {rmse_3:.2f}")
print(f"Single Layer LSTM (lag=24): {rmse_24:.2f}")
print(f"Bidirectional LSTM (lag=24): {rmse_bi:.2f}")
    
# Plot comparison
plt.figure(figsize=(14, 6))
plt.plot(y_test_24, label='Actual')
plt.plot(predictions_24, label='Single Layer LSTM (lag=24)')
plt.plot(predictions_bi, label='Bidirectional LSTM (lag=24)')
plt.title('Model Comparison on Test Set')
plt.xlabel('Time Step')
plt.ylabel('Power (MW)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

if __name__ == "__main__":
    main()